Copyright 2020 Natasha A. Sahr, Andrew M. Olney and made available under [CC BY-SA](https://creativecommons.org/licenses/by-sa/4.0) for text and [Apache-2.0](http://www.apache.org/licenses/LICENSE-2.0) for code.


# Ridge and Lasso Regression: L1 and L2 penalization

## Regularization

Up to this point, we've focused on relatively small numbers of predictors in our models.
When we have datasets with large numbers of predictors, we need to think about new techniques to deal with the additional complexity.
Part of the reason is that our highly manual methods no longer scale well.
Imagine if you had to make and evaluate plots for 1000 variables!
The other part is that once we have many variables, the chances of them interacting with each other in very complicated ways gets increasingly larger.

We talked about one "bad" kind of interaction before, multicolinearity.
Multicolinearity occurs when two variables mostly measure the same thing.
The problem with multicolinearity in linear models is that the variables involved will no longer have unique solutions for their estimated coefficients.
What this means in practice is that multicolinearity is a small, but manageable problem for small datasets, but multicolinearity becomes a very serious problem for large datasets, at least for linear models, which arguably are the most important models in science.

Today we will talk about two methods that address the complexity of having many variables, including multicolinearity.
Both of these methods use a "big idea" in data science called **regularization**.
The idea behind regularization is that you **penalize** complex models in favor of simpler ones.
These simpler models use fewer variables, making them easier to understand.
If you penalization is set up in the right way, the simpler models can also avoid multicolinearity problems.
Today we will focus on ridge and lasso regression, but it is important to remember that many other models use similar regularization techniques. 
Once you know to look for it, you will start to see it everywhere!

## What you will learn

In the sections that follow, you will learn about ridge and lasso regularization and how they can help us assess a large number of variables in for candidacy in regression models by penalizing variables that don't contribute a large effect in the variability of the outcome. We will study the following:

- Ridge regression with the L2 penalty
- Lasso regression with the L1 penalty
- Assessing model accuracy
- Comparing regularized models 
- Selection of the tuning parameter $\lambda$

## When to use regularization/penalization in regression

Regularization is a general strategy that applies a penalty in the optimization of a regression model. With the correct tuning parameter selection, it will prevent overfitting a model to a particular dataset and improve the potential for generalization to new datasets.
Regularization becomes particularly important in regression where there are large numbers of predictors because it can mitigate multicollinearity and cause shrinkage (for L2) or encourage sparsity (for L1) of the coefficients for variables that contribute less to the prediction of the outcome.

## Vanilla logistic regression

Let's start by applying logistic regression to some breast cancer data.
This model will serve as a baseline for comparison to the ridge and lasso models that come later.
We're going to get the breast cancer data from `sklearn` instead of loading a CSV.
Libraries like `sklearn` frequently come with their own datasets for demonstration purposes.

### Load data

The [data](https://scikit-learn.org/stable/datasets/index.html#breast-cancer-dataset) consists of the following variables as mean, standard error, and "worst" (mean of three largest variables) collected by digital imagery of a biopsy.

| Variable | Type | Description |
|:-------|:-------|:-------|
|radius | Ratio | mean of distances from center to points on the perimeter|
|texture | Ratio | standard deviation of gray-scale values|
|perimeter | Ratio | perimeter of cancer|
|area | Ratio | area of cancer|
|smoothness | Ratio | local variation in radius lengths|
|compactness | Ratio |  perimeter^2 / area - 1.0|
|concavity | Ratio |  severity of concave portions of the contour|
|concave points | Ratio |  number of concave portions of the contour|
|symmetry | Ratio | symmetry of cancer|
|fractal dimension | Ratio | "coastline approximation" - 1|
| class | Nominal (binary) | malignant (1) or benign (0)

<div style="text-align:center;font-size: smaller">
    <b>Source:</b> This dataset was taken from the <a href="https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)">UCI Machine Learning Repository library
    </a>
</div>
<br>

We want to predict the presence/absence of cancer, so it makes sense to use logistic regression rather than linear regression.
Ridge and lasso penalties work the same way in both kinds of regression.

First, import libraries for dataframes and to load the dataset: 

- `import pandas as pd`
- `import sklearn.datasets as datasets`

Next we need to do some conversion to put the `sklearn` data into a dataframe, which is the form we are most comfortable with:

- Create variable `cancer_sklearn`
- Set it to `with datasets do load_breast_cancer using`

The next step is to put this `sklearn` data into a dataframe:

- Create variable `dataframe`
- Set it to `with pd create DataFrame using` a list containing
    - `from cancer_sklearn get data`
    - freestyle `columns=` followed by `from cancer_sklearn get feature_names`

You can use this approach to put any `sklearn` format dataset into a dataframe.

Because this data is too "easy", we need to make it more complicated to really show the benefits of ridge and lasso.
Remember that multicolinearity is bad? 
Let's make it artificially multicolinear by duplicating the columns in the dataframe so that we have four side by side:

- Set `dataframe` to `with pd do concat using` a list containing
    - A list containing
        - `dataframe` 
        - `dataframe` 
        - `dataframe` 
        - `dataframe` 
    - freestyle `axis=1`
    
The axis tells it to stack horizontally rather than vertically.

`sklearn` stores the predictors and the target (outcome) variable separately, so we need to `assign` it to the dataframe:

- Set `dataframe` to `with dataframe do assign using` a list containing
    - freestyle `Target=` followed by `from cancer_sklearn get target`
- `dataframe` (to display)

As you can see, we're now working with 120 predictor variables instead of the original 30.
It's a lot more than we'd like to have to examine manually. 

### Explore data

Based on the earlier discussion, you might guess that this dataset has a problem with multicolinearity.
And you'd be right!

Let's make a quick heatmap to show this:

- `import plotly.express as px`

And now a "one line" heatmap:

- `with px do imshow using` as list containing
    - `with dataframe do corr using`
    - A freestyle block **with a notch on the right** containing `x=`, connected to `from dataframe get columns`
    - A freestyle block **with a notch on the right** containing `y=`, connected to `from dataframe get columns`

Anything light orange to yellow could give us positive colinearity problems, and anything dark purple to indigo could give us negative colinearity problems.

Depending on the size of your screen, `plotly` may only show every second or third variable name.
You can use the Zoom tool to explore this correlation matrix more closely.

## Prepare train/test sets

We need to split the dataframe into `X`, our predictors, and `Y`, our target variable (breast cancer positive).

Do the imports for splitting:

- `import sklearn.model_selection as model_selection`

Create `X` by dropping the label from the dataframe:

- Create variable `X`
- Set it to `with dataframe do drop using` a list containing
    - freestyle `columns=["Target"]`
- `X` (to display)

Create `Y` by pulling just `Target` from the dataframe:

- Create variable `Y`
- Set it to `dataframe [ ] ` containing the following in a list
    - `"Target"`
- `Y` (to display)

Now do the splits:

- Create variable `splits`
- Set it to `with model_selection do train_test_split using` a list containing
    - `X`
    - `Y`
    - freestyle `random_state=2` (this will make your random split the same as mine)
    
**Notice we did not specify a test size; `sklearn` will use .25 by default**.

### Model 1: Logistic regression

Let's do something we already suspect is not a great idea: regular logistic regression.

First, the imports for regression, evaluation, preprocessing, and pipelines:

- `import sklearn.linear_model as linear_model`
- `import sklearn.metrics as metrics`
- `import numpy as np`
- `import sklearn.preprocessing as pp`
- `import sklearn.pipeline as pipe`

#### Training

We're going to make a pipeline so we can scale and train in one step:

- Create variable `std_clf`
- Set it to `with pipe do make_pipeline using` a list containing
    - `with pp create StandardScaler using`
    - `with linear_model create LogisticRegression using` a list containing
        - freestyle `penalty="none"`

**The "none" penalty here is important because `sklearn` uses a ridge penalty by default.**

We can treat the whole pipeline as a classifier and call `fit` on it:

-  `with std_clf do fit using` a list containing
    - `in list splits get # 1` (this is Xtrain)
    - `with np do ravel using` a list containing
        - `in list splits get # 3` (this is Ytrain)

Now we can get predictions from the model for our test data:

- Create variable `predictions`
- Set it to `with std_clf do predict using` a list containing
    - `in list splits get # 2` (this is Xtest)
- `predictions` (to display)

#### Evaluation

To get the accuracy:

- `print create text with`
    - "Accuracy:"
    - `with metrics do accuracy_score using` a list containing
        - `in list splits get # 4`  (this is `Ytest`)
        - `predictions`

To get precision, recall, and F1:

- `print with metrics do classification_report using` a list containing
    - `in list splits get # 4`  (this is `Ytest`)
    - `predictions`

## Regression with a Ridge (L2) Penalty

In ridge regression, also known as L2 penalization, the cost function is altered by adding a penalty equivalent to the square of the magnitude of the coefficients. This is equivalent to saying: for some $c > 0$, $\sum_{j=0}^p \beta_j^2 < 0$ for coefficients $\beta_j, j=1,\dots,p$. 

The cost function for ridge regression is

$$\sum_{i=1}^N (y_i-\hat{y_i})^2 = \sum_{i=1}^N (y_i - \sum_{j=0}^p \beta_i x_{ij})^2 + \lambda \sum_{j=0}^p \beta_j^2$$

When $\lambda = 0$, we have is a linear regression model.

The $\lambda$ regularizes the coefficients so the optimization function is penalized if the coefficients are large. This type of penalization leads to coefficients close to, but not exactly, zero. This feature of ridge regression shrinks the coefficients allowing for a reduction of model complexity and multicollinearity.

### Model 2: Logistic ridge regression (C=.75)

Adding a ridge penalty is almost *exactly* like the model we did before.
There are two differences:

- penalty="l2"
- C = .75

The ridge penalty is an l2 penalty (because it's squared).
The `C` value is the **amount** of the penalty.
In `sklearn` it is inverted, so smaller numbers mean more penalty.

**Do yourself a favor and copy the blocks you've already done. You can save your notebook, right click it in the file browser, select "Duplicate", and then open that copy in 3 pane view by dragging the tab to the center right. Make sure you change variable names as directed.**

**Here's how to duplicate:**



**Here's 3 pane view:**



#### Training

- Create variable `std_clf_ridge75`
- Set it to `with pipe do make_pipeline using` a list containing
    - `with pp create StandardScaler using`
    - `with linear_model create LogisticRegression using` a list containing
        - freestyle `penalty="l2"`
        - freestyle `C=0.75`

We can treat the whole pipeline as a classifier and call `fit` on it:

-  `with std_clf_ridge75 do fit using` a list containing
    - `in list splits get # 1` (this is Xtrain)
    - `with np do ravel using` a list containing
        - `in list splits get # 3` (this is Ytrain)

Now we can get predictions from the model for our test data:

- Create variable `predictions_ridge75`
- Set it to `with std_clf_ridge75 do predict using` a list containing
    - `in list splits get # 2` (this is Xtest)

#### Evaluation

To get the accuracy:

- `print create text with`
    - "Accuracy:"
    - `with metrics do accuracy_score using` a list containing
        - `in list splits get # 4`  (this is `Ytest`)
        - `predictions_ridge75`

To get precision, recall, and F1:

- `print with metrics do classification_report using` a list containing
    - `in list splits get # 4`  (this is `Ytest`)
    - `predictions_ridge75`

We went from .923 accuracy and .92 weighted avg f1 to .965 accuracy and .97 weighted average f1, just by using the ridge penalty.

### Model 3: Logistic ridge regression (C=.25)

This model is the same as model 2 but with a different ridge penalty:

- penalty="l2"
- C = .25

#### Training

- Create variable `std_clf_ridge25`
- Set it to `with pipe do make_pipeline using` a list containing
    - `with pp create StandardScaler using`
    - `with linear_model create LogisticRegression using` a list containing
        - freestyle `penalty="l2"`
        - freestyle `C=0.25`

We can treat the whole pipeline as a classifier and call `fit` on it:

-  `with std_clf_ridge25 do fit using` a list containing
    - `in list splits get # 1` (this is Xtrain)
    - `with np do ravel using` a list containing
        - `in list splits get # 3` (this is Ytrain)

Now we can get predictions from the model for our test data:

- Create variable `predictions_ridge25`
- Set it to `with std_clf_ridge25 do predict using` a list containing
    - `in list splits get # 2` (this is Xtest)

#### Evaluation

To get the accuracy:

- `print create text with`
    - "Accuracy:"
    - `with metrics do accuracy_score using` a list containing
        - `in list splits get # 4`  (this is `Ytest`)
        - `predictions_ridge25`

To get precision, recall, and F1:

- `print with metrics do classification_report using` a list containing
    - `in list splits get # 4`  (this is `Ytest`)
    - `predictions_ridge25`

We went from .965 accuracy and .97 weighted average f1 to .972 accuracy and .97 weighted average f1, again just by using the ridge penalty but with a greater penalty.

### Comparing Ridge models

Let's plot the coefficients of models 1 to 3 to show the effect of the ridge penalty on the coefficents.
Remember, the penalty *shrinks* coefficients.

Do the imports for plotting with layers:

- `import plotly.graph_objects as go`

and we need to create dummy x-axis for our coefficents:

- Create variable `dummyX`
- Set it to `with np do linspace using` a list containing:
    - `1`
    - `length of` `from dataframe get columns` - `1`
    - `length of` `from dataframe get columns` - `1`
    
**That last part is supposed to be twice.
Since there is one target column and the rest are predictors, we subtract 1.**

Create an empty figure:
    
- Create `fig`
- Set it to `with go create Figure using`

Add three scatterplots to `fig`:

- `with fig do add_scatter using`
    - freestyle `x=dummyX`
    - freestyle `y=np.ravel(std_clf[1].coef_)`
    - freestyle `name='Logistic Regression'`
    - freestyle `mode='markers'`
    - freestyle `marker=dict(color='blue', opacity=0.25, size=30)`
    
**For the next two, copy the first and make small changes.**

- `with fig do add_scatter using`
    - freestyle `x=dummyX`
    - freestyle `y=np.ravel(std_clf_ridge75[1].coef_)`
    - freestyle `name='Logistic Ridge Regression C=.75'`
    - freestyle `mode='markers'`
    - freestyle `marker=dict(color='green', opacity=0.50, size=15)`
    
    
- `with fig do add_scatter using`
    - freestyle `x=dummyX`
    - freestyle `y=np.ravel(std_clf_ridge25[1].coef_)`
    - freestyle `name='Logistic Ridge Regression C=.25'`
    - freestyle `mode='markers'`
    - freestyle `marker=dict(color='red', opacity=0.75, size=8)`

To get a sense of the shrinkage, use the magnifying glass tool in `plotly` to zoom in until the y axis is about -2 to 2.
Then you can see that C=.25 penalty even more tightly shrunk that C=.75.

## Regression with a Lasso (L1) Penalty

In lasso regression, also known as L1 penalization, the cost function is altered by adding a penalty equivalent to the absolute value of the magnitude of the coefficients. This is equivalent to saying: for some $c > 0$, $|\beta_j| < 0$ for coefficients $\beta_j, j=1,\dots,p$. 

The cost function for lasso regression is

$$\sum_{i=1}^N (y_i-\hat{y_i})^2 = \sum_{i=1}^N (y_i - \sum_{j=0}^p \beta_i x_{ij})^2 + \lambda \sum_{j=0}^p |\beta_j|$$

When $\lambda = 0$, we have is a linear regression model.

The $\lambda$ regularizes the coefficients so the optimization function is penalized if the coefficients are large. This type of penalization leads to exactly zero coefficients. This feature of lasso regression shrinks the coefficients allowing for a reduction of model complexity and multicollinearity and allows use to perform feature selection.

### Model 4: Logistic lasso regression (C=.75)

Adding a lasso penalty is almost *exactly* like the model we did before.
There are three differences:

- penalty="l1"
- C = .75
- solver="liblinear"

The lasso penalty is an l1 penalty (because it's absolute value). 
The "solver" is the algorithm that implements the l1 penalty.

#### Training

- Create variable `std_clf_lasso75`
- Set it to `with pipe do make_pipeline using` a list containing
    - `with pp create StandardScaler using`
    - `with linear_model create LogisticRegression using` a list containing
        - freestyle `penalty="l1"`
        - freestyle `C=0.75`
        - freestyle `solver="liblinear"`

We can treat the whole pipeline as a classifier and call `fit` on it:

-  `with std_clf_lasso75 do fit using` a list containing
    - `in list splits get # 1` (this is Xtrain)
    - `with np do ravel using` a list containing
        - `in list splits get # 3` (this is Ytrain)

Now we can get predictions from the model for our test data:

- Create variable `predictions_lasso75`
- Set it to `with std_clf_lasso75 do predict using` a list containing
    - `in list splits get # 2` (this is Xtest)

#### Evaluation

To get the accuracy:

- `print create text with`
    - "Accuracy:"
    - `with metrics do accuracy_score using` a list containing
        - `in list splits get # 4`  (this is `Ytest`)
        - `predictions_lasso75`

To get precision, recall, and F1:

- `print with metrics do classification_report using` a list containing
    - `in list splits get # 4`  (this is `Ytest`)
    - `predictions_lasso75`

Interestingly, model 4 is the same as model 2. Both are .965 accuracy and .97 weighted average f1.

### Model 5: Logistic lasso regression (C=.25)

This model is the same as model 4 but with a different ridge penalty:

- penalty="l1"
- C = .25
- solver="liblinear"

#### Training

- Create variable `std_clf_lasso25`
- Set it to `with pipe do make_pipeline using` a list containing
    - `with pp create StandardScaler using`
    - `with linear_model create LogisticRegression using` a list containing
        - freestyle `penalty="l1"`
        - freestyle `C=0.25`
        - freestyle `solver="liblinear"`

We can treat the whole pipeline as a classifier and call `fit` on it:

-  `with std_clf_lasso25 do fit using` a list containing
    - `in list splits get # 1` (this is Xtrain)
    - `with np do ravel using` a list containing
        - `in list splits get # 3` (this is Ytrain)

Now we can get predictions from the model for our test data:

- Create variable `predictions_lasso25`
- Set it to `with std_clf_lasso25 do predict using` a list containing
    - `in list splits get # 2` (this is Xtest)

#### Evaluation

To get the accuracy:

- `print create text with`
    - "Accuracy:"
    - `with metrics do accuracy_score using` a list containing
        - `in list splits get # 4`  (this is `Ytest`)
        - `predictions_lasso25`

To get precision, recall, and F1:

- `print with metrics do classification_report using` a list containing
    - `in list splits get # 4`  (this is `Ytest`)
    - `predictions_lasso25`

Model 5 (accuracy .979 and weighted avg f1 .98) is slightly better than model 3 (.972 accuracy and .97 weighted average f1).
Additionally, lasso has done something that ridge didn't do: it has shrunk many coefficients to zero.
So it's actually pretty amazing that lasso is slightly better than ridge after so many variables have been removed.
To see which ones, run the cell below:

In [112]:
m5coefficients = pd.DataFrame( {"variable":X.columns, "coefficient":np.ravel(std_clf_lasso25[1].coef_) })
print(m5coefficients.to_string())
print( 'Variables removed (zero coefficient):' , len( m5coefficients[m5coefficients['coefficient']==0.0] ) )

                    variable  coefficient
0                mean radius     0.000000
1               mean texture     0.000000
2             mean perimeter     0.000000
3                  mean area     0.000000
4            mean smoothness     0.000000
5           mean compactness     0.000000
6             mean concavity    -0.040312
7        mean concave points    -0.003495
8              mean symmetry     0.000000
9     mean fractal dimension     0.000000
10              radius error    -0.001453
11             texture error     0.000000
12           perimeter error     0.000000
13                area error     0.000000
14          smoothness error     0.000000
15         compactness error     0.000000
16           concavity error     0.000000
17      concave points error     0.000000
18            symmetry error     0.000000
19   fractal dimension error     0.000000
20              worst radius     0.000000
21             worst texture    -0.415234
22           worst perimeter     0

We started with 120 and zeroed out 84. 
Obviously many of the variables remaining are duplicates; handling these is a more complex topic!

### Comparing Lasso models

As before, let's plot the coefficients of models 1, 4, and 5 to show the effect of the lasso penalty on the coefficients.

Create an empty figure:

- Set `fig` to `with go create Figure using`

Add three scatterplots to `fig`:

- `with fig do add_scatter using`
    - freestyle `x=dummyX`
    - freestyle `y=np.ravel(std_clf[1].coef_)`
    - freestyle `name='Logistic Regression'`
    - freestyle `mode='markers'`
    - freestyle `marker=dict(color='blue', opacity=0.25, size=30)`
    
**For the next two, copy the first and make small changes.**

- `with fig do add_scatter using`
    - freestyle `x=dummyX`
    - freestyle `y=np.ravel(std_clf_lasso75[1].coef_)`
    - freestyle `name='Logistic Lasso Regression C=.75'`
    - freestyle `mode='markers'`
    - freestyle `marker=dict(color='green', opacity=0.50, size=15)`
    
    
- `with fig do add_scatter using`
    - freestyle `x=dummyX`
    - freestyle `y=np.ravel(std_clf_lasso25[1].coef_)`
    - freestyle `name='Logistic Lasso Regression C=.25'`
    - freestyle `mode='markers'`
    - freestyle `marker=dict(color='red', opacity=0.75, size=8)`

Again, us the magnifying glass tool in `plotly` to zoom in until the y axis is about -1 to 1, and notice how many coefficients are zero for C=.75 and C=.25.

We can count how many are zero in each case as well:

- `print create text with`
    - `"C=.75:"`
    - `with np do sum using` freestyle `std_clf_lasso75[1].coef_` = `0`
- `print create text with`
    - `"C=.25:"`
    - `with np do sum using` freestyle `std_clf_lasso25[1].coef_` = `0`

Even a small amount of penalization, in this case, removed a lot of variables.
Lasso is a great way to simplify your model!

## Choosing $\lambda$ (AKA C)

As we've seen, different values of the penalty parameter $\lambda$ have different effects on our accuracy and other performance metrics.
So how do we choose  $\lambda$?
There are a number of different methods of finding a **metaparameter** like  $\lambda$, and these methods are fairly general and apply to other problems we've seen before, like choosing the optimal number of clusters or the optimal number of nearest neighbors.
We will revisit this idea in the future. 